## Simulate data CDD

This script is used to generate simulated data for the CDD task. Then we can run parameter_recovery to estimate those parameters from the simulated data


In [1]:
# Built-in/Generic Imports
import os

# Libs
import numpy as np
import pandas as pd
from scipy.stats import bernoulli

In [2]:
# Ben's parameter range

min_k = 0.00000114
max_k = 0.768891643
min_b = 0.002337451 #min among non-exploded models
max_b = 0.113426498

n = 11

lilks = np.arange(min_k, max_k, (max_k-min_k)/n)
lilbs = np.arange(min_b, max_b, (max_b-min_b)/n)
np.log(lilks)

array([-13.6844823 ,  -2.66068567,  -1.96754665,  -1.56208426,
        -1.27440354,  -1.05126081,  -0.86893979,  -0.7147895 ,
        -0.5812584 ,  -0.46347559,  -0.35811526])

In [3]:

### Defining the grids, and functions to make sets as dataframes: choice_set,param_set,response_set

def set_grids(nb_kappa=10,nb_gamma=10):
    grid_design = {
        # now: time_null = 0
        'cdd_immed_wait': [0],
        # time to get reward (delay_wait)
        'cdd_delay_wait': [2, 7, 15, 36, 72, 108, 144, 180],
        # immediate reward from experiment: $2, $10, $20 ... fixed to $10
        'cdd_immed_amt': [5, 10],
        # reward amount set to vary according to experiment
        'cdd_delay_amt': [11, 15, 23, 30, 41, 50, 65]
    }
    grid_param = {
        # 50 points on [10^-5, ..., 1] in a log scale
        'kappa_gt': np.logspace(-7, 1.5, nb_kappa, base=np.exp(1)),
        # 10 points on (0, 5] in a linear scale
        'gamma_gt': np.linspace(0, 3, nb_gamma+1)[1:]
    }
    return grid_design,grid_param

def insert_var(grid=[],var_nb=0,list_var=[]):
    if not var_nb:
        grid = list_var
    elif var_nb==1:
        grid = [[g]+[i] for g in grid for i in list_var]
    else:
        grid = [g+[i] for g in grid for i in list_var]
    return grid

def make_grid(grid_values):
    grid = []
    labels = grid_values.keys()
    for var_nb,row in enumerate(grid_values.items()):
        grid = insert_var(grid=grid,var_nb=var_nb,list_var=row[1])
    grid_df = pd.DataFrame(grid,columns=labels)
    return grid_df


In [4]:

def simulate_response(design,PARAM_TRUE):
    # Calculate the probability to choose a variable option
    tn, tr, vn, vr = (design['cdd_immed_wait'], design['cdd_delay_wait'],design['cdd_immed_amt'], design['cdd_delay_amt'])
    kappa, gamma = PARAM_TRUE['kappa_gt'], PARAM_TRUE['gamma_gt']
    
    SV_null = vn * (1. / (1 + kappa * tn))
    SV_reward = vr * (1. / (1 + kappa * tr))
    p_obs = 1. / (1 + np.exp(-gamma * (SV_reward - SV_null)))

    # Randomly sample a binary choice response from Bernoulli distribution
    return float(bernoulli.rvs(p_obs))

def generate_response(design_df,PARAM_TRUE):
    response= []
    df = pd.DataFrame([])
    for index, design in design_df.iterrows():
        response += [simulate_response(design,PARAM_TRUE)]
    df['cdd_choice'] = response
    df['cdd_trial_type'] = 'task'
    df['cdd_confkey'] = 1.0
    return df


In [5]:
# can set resolution (number of parameter) for kappa and gamma
grid_design,grid_param = set_grids(nb_kappa=10,nb_gamma=10)

design_df = make_grid(grid_design)
fn = os.path.join('simul','design_set.csv')
print('Saving design to : {}'.format(fn))
design_df.to_csv(fn)

param_df = make_grid(grid_param)
fn = os.path.join('simul','ground_truth.csv')
print('Saving ground truth parameters to : {}'.format(fn))
param_df.to_csv(fn)

for index, PARAM_TRUE in param_df.iterrows():
    response_df = generate_response(design_df,PARAM_TRUE)
    subj_dir = os.path.join('simul','split','p{0:04d}'.format(index),'cdd')
    if not os.path.exists(subj_dir):
        os.makedirs(subj_dir)
    fn = os.path.join(subj_dir,'p{0:04d}_cdd.csv'.format(index))
    print('Saving response to : {}'.format(fn))
    df = pd.concat([design_df,response_df],axis=1)
    df.to_csv(fn)


Saving design to : simul/design_set.csv
Saving ground truth parameters to : simul/ground_truth.csv
Saving response to : simul/split/p0000/cdd/p0000_cdd.csv
Saving response to : simul/split/p0001/cdd/p0001_cdd.csv
Saving response to : simul/split/p0002/cdd/p0002_cdd.csv
Saving response to : simul/split/p0003/cdd/p0003_cdd.csv
Saving response to : simul/split/p0004/cdd/p0004_cdd.csv
Saving response to : simul/split/p0005/cdd/p0005_cdd.csv
Saving response to : simul/split/p0006/cdd/p0006_cdd.csv
Saving response to : simul/split/p0007/cdd/p0007_cdd.csv
Saving response to : simul/split/p0008/cdd/p0008_cdd.csv
Saving response to : simul/split/p0009/cdd/p0009_cdd.csv
Saving response to : simul/split/p0010/cdd/p0010_cdd.csv
Saving response to : simul/split/p0011/cdd/p0011_cdd.csv
Saving response to : simul/split/p0012/cdd/p0012_cdd.csv
Saving response to : simul/split/p0013/cdd/p0013_cdd.csv
Saving response to : simul/split/p0014/cdd/p0014_cdd.csv
Saving response to : simul/split/p0015/cdd/p00